In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv("asset_1.csv")

In [4]:
df.head()

,serial_num,price,alpha
0,0,18105.300781,0.000000
1,1,18398.960605,-0.630291
2,2,18204.939538,-0.123420
3,3,18339.357782,-0.240239
4,4,18578.084798,-0.612625


In [5]:
df['position'] = 0
build_threshold = 0.6
liquidate_threshold = 0.2
current_position = 0

for i in range(1, len(df)):
    if current_position == 0:
        if df.loc[i, 'alpha'] >= build_threshold:
            current_position = 1 
        elif df.loc[i, 'alpha'] <= -build_threshold:
            current_position = -1
    elif current_position == 1:
        if df.loc[i, 'alpha'] <= liquidate_threshold:
            current_position = 0
    elif current_position == -1:
        if df.loc[i, 'alpha'] >= -liquidate_threshold:
            current_position = 0 
    
    df.loc[i, 'position'] = current_position

df.loc[len(df)-1, 'position'] = 0

df.to_csv('asset_data_with_positions.csv', index=False)
df.head()


,serial_num,price,alpha,position
0,0,18105.300781,0.000000,0
1,1,18398.960605,-0.630291,-1
2,2,18204.939538,-0.123420,0
3,3,18339.357782,-0.240239,0
4,4,18578.084798,-0.612625,-1


In [6]:
df['trade'] = df['position'].diff().fillna(0)
df['price_change'] = df['price'].diff().fillna(0)
df['pnl'] = 0

for i in range(1, len(df)):
    if df.loc[i, 'trade'] != 0:
        df.loc[i, 'pnl'] = -df.loc[i, 'trade'] * df.loc[i, 'price']
    df.loc[i, 'pnl'] += df.loc[i, 'position'] * df.loc[i, 'price_change']

total_pnl = df['pnl'].sum()
print(f"Total PnL: {total_pnl}")

df.to_csv('asset_data_with_pnl.csv', index=False)
df.head()


Total PnL: 37260.16158119356


,serial_num,price,alpha,position,trade,price_change,pnl
0,0,18105.300781,0.000000,0,0.0,0.000000,0.000000
1,1,18398.960605,-0.630291,-1,-1.0,293.659824,18105.300781
2,2,18204.939538,-0.123420,0,1.0,-194.021067,-18204.939538
3,3,18339.357782,-0.240239,0,0.0,134.418244,0.000000
4,4,18578.084798,-0.612625,-1,-1.0,238.727016,18339.357782


In [7]:
def calculate_pnl(build_threshold, liquidate_threshold, df):
    df['position'] = 0
    current_position = 0
    for i in range(1, len(df)):
        if current_position == 0:
            if df.loc[i, 'alpha'] >= build_threshold:
                current_position = 1
            elif df.loc[i, 'alpha'] <= -build_threshold:
                current_position = -1
        elif current_position == 1:
            if df.loc[i, 'alpha'] <= liquidate_threshold:
                current_position = 0
        elif current_position == -1:
            if df.loc[i, 'alpha'] >= -liquidate_threshold:
                current_position = 0
        df.loc[i, 'position'] = current_position
    df.loc[len(df)-1, 'position'] = 0
    df['trade'] = df['position'].diff().fillna(0)
    df['price_change'] = df['price'].diff().fillna(0)
    df['pnl'] = 0
    for i in range(1, len(df)):
        if df.loc[i, 'trade'] != 0:
            df.loc[i, 'pnl'] = -df.loc[i, 'trade'] * df.loc[i, 'price']
        df.loc[i, 'pnl'] += df.loc[i, 'position'] * df.loc[i, 'price_change']
    return df['pnl'].sum()
build_thresholds = np.arange(0.1, 1.0, 0.1)
liquidate_thresholds = np.arange(0.1, 1.0, 0.1)

best_pnl = float('-inf')
best_thresholds = (0, 0)
for build_threshold in build_thresholds:
    for liquidate_threshold in liquidate_thresholds:
        pnl = calculate_pnl(build_threshold, liquidate_threshold, df.copy())
        if pnl > best_pnl:
            best_pnl = pnl
            best_thresholds = (build_threshold, liquidate_threshold)

print(f"Optimal Build Threshold: {best_thresholds[0]}")
print(f"Optimal Liquidate Threshold: {best_thresholds[1]}")
print(f"Best PnL: {best_pnl}")


Optimal Build Threshold: 0.30000000000000004
Optimal Liquidate Threshold: 0.1
Best PnL: 49126.811733655326
